In [192]:
#pip install tensorflow

# Importing libraries and dataset

In [19]:
import numpy as np
import pandas as pd
import tensorflow as tf

# Check tensor flow version 

In [20]:
tf.__version__

'2.15.0'

# Read data file 

In [21]:
file_name = r'C:\Users\Maryam\Desktop\Job applications\Portfolio\Portfolio_Projects\Churn_Modelling.csv'
dataset = pd.read_csv(file_name)
# from column 3 and not the last column 
x = dataset.iloc[:, 3:-1].values
# only the last column  
y = dataset.iloc[:, -1].values 

# Part 1 - Data Preprocessing

## Encoding categorical data

### Label enconding the "Gender" column 

In [22]:
import sklearn 
from sklearn.preprocessing import LabelEncoder
# to transform the gender into 0 and 1 
le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:, 2])


In [23]:
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


## one hot encoding the "Geography" column

In [24]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder() ,[1])], remainder='passthrough')
x = np.array(ct.fit_transform(x))


print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [25]:
x.shape

(10000, 12)

In [26]:
y.shape 

(10000,)

## Splitting the dataset into the training and test sets 

In [27]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [28]:
x_train.shape

(8000, 12)

In [29]:
y_train.shape 

(8000,)

## Feature Scaling 

In [30]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train= sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)



# Part2 - Building the ANN

## Initializing the ANN

In [31]:
ann = tf.keras.models.Sequential()


## Adding the input layer and the first hidden layer

In [32]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))


## Adding the second input layer

In [33]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))


## Adding the output layer 

In [35]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Part 3 - Training the ANN

## Compiling the ANN

In [36]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Training the ANN on the training set 

In [37]:
ann.fit(x_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100


250/250 [==============================] - 1s 1ms/step - loss: 0.6257 - accuracy: 0.7960
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5384 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4926 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4692 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4545 - accuracy: 0.7960
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4414 - accuracy: 0.7960
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4311 - accuracy: 0.7960
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4237 - accuracy: 0.7960
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4178 - accuracy: 0.7960
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4128 - accu

# Part 4 - Making the predictions and evaluating the model 

## Predicting the result of a single observation 

In [177]:
# predict if the following customer will leave the bank
# Geography: France (1,0,0)
# Credit Scaore: 600
# Gender: Male (1)
# Age: 40 
# Tenure: 3 
# Balance: 60000
# Number of Product: 2
# Have a credit card: Yes (1)
# Active customer: Yes (1)
# Estimated Salary: 50000
ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))

1/1 [==============================] - 0s 178ms/step


array([[0.03273582]], dtype=float32)

In [179]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))>0.5)

1/1 [==============================] - 0s 24ms/step
[[False]]


## Predicting the test set results 

In [184]:
y_pred = ann.predict(x_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)), 1))

 1/63 [..............................] - ETA: 1s

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


## Making the confusion matrix 

In [191]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1527   68]
 [ 212  193]]


0.86